In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import regularizers
from keras.regularizers import l1,l2
from keras.utils.np_utils import to_categorical

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 100
num_classes = 10
epochs =250
l = 40
growth_rate=12
bottleneck=False
num_filter = 12
compression = 0.5
dropout_rate = 0.2
strides=(2, 2)
dilation_rate=(1, 1)

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Dense Block
def add_denseblock(input, num_filter, dropout_rate,growth_rate):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',
                                  kernel_regularizer=regularizers.l2(0.0001))(relu)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',
                           kernel_regularizer=regularizers.l2(0.0001))(relu)
       
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter, dropout_rate ,growth_rate):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
   
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',
                              kernel_regularizer=regularizers.l2(0.0001))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
num_filter = 26
dropout_rate = 0.2

growth_rate=12
compression = 0.5
input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
l=6
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate,growth_rate)

First_Transition = add_transition(First_Block, num_filter, dropout_rate,growth_rate)
l=12
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate,growth_rate)

Second_Transition = add_transition(Second_Block, num_filter, dropout_rate,growth_rate)

l=24

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate,growth_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate,growth_rate)
l=16
Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate,growth_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 26)   702         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 26)   104         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 26)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_3 (

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=sgd(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [12]:
def step_decay(epochs):
    learning_rate=0.1
    if epochs >= 100 and epochs <150:
        learning_rate=0.01 
        print('lr_changed to 0.01')
    if epochs >= 150 and epochs <170:
        learning_rate=0.001 
        print('lr_changed to 0.001')
    if epochs >= 170 and epochs <200:
        learning_rate=0.0005
        print('lr_changed to 0.0005')
    if epochs >= 200 and epochs <230:
        learning_rate=0.0001
        print('lr_changed to 0.0001')
    if epochs >= 230:
        learning_rate=0.00001
        print('lr_changed to 0.00001')
        
 
    return float(learning_rate)
learning_rate = LearningRateScheduler(step_decay)   


#model_checkpoint= ModelCheckpoint('drive/ColabNotebooks/older/weights/4A1.h5', monitor="val_acc", save_best_only=True,save_weights_only=True, verbose=1)
                                           
                                           
                                           

In [13]:
#data augmentation
datagen = ImageDataGenerator( rotation_range=15,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True) 
datagen.fit(x_train)

In [14]:

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                   epochs = epochs, steps_per_epoch=x_train.shape[0]//batch_size ,validation_data = (x_test, y_test),callbacks=[learning_rate],
                   verbose = 1 )
                  

Epoch 1/250
500/500 [==============================] - 67s 134ms/step - loss: 2.2240 - acc: 0.2738 - val_loss: 3.5865 - val_acc: 0.1459
Epoch 2/250
500/500 [==============================] - 53s 106ms/step - loss: 1.8063 - acc: 0.3896 - val_loss: 2.8966 - val_acc: 0.2712
Epoch 3/250
500/500 [==============================] - 55s 111ms/step - loss: 1.5795 - acc: 0.4735 - val_loss: 1.6205 - val_acc: 0.4729
Epoch 4/250
500/500 [==============================] - 53s 105ms/step - loss: 1.4120 - acc: 0.5385 - val_loss: 2.1119 - val_acc: 0.4304
Epoch 5/250
500/500 [==============================] - 53s 105ms/step - loss: 1.2842 - acc: 0.5846 - val_loss: 1.5247 - val_acc: 0.5391
Epoch 6/250
500/500 [==============================] - 53s 106ms/step - loss: 1.1967 - acc: 0.6162 - val_loss: 1.6536 - val_acc: 0.5421
Epoch 7/250
500/500 [==============================] - 53s 107ms/step - loss: 1.1238 - acc: 0.6456 - val_loss: 1.7611 - val_acc: 0.5524
Epoch 8/250
500/500 [===========================

Epoch 61/250
500/500 [==============================] - 52s 104ms/step - loss: 0.6779 - acc: 0.8401 - val_loss: 1.1093 - val_acc: 0.7408
Epoch 62/250
500/500 [==============================] - 55s 109ms/step - loss: 0.6762 - acc: 0.8411 - val_loss: 0.8819 - val_acc: 0.7838
Epoch 63/250
500/500 [==============================] - 52s 103ms/step - loss: 0.6723 - acc: 0.8421 - val_loss: 1.1473 - val_acc: 0.7270
Epoch 64/250
500/500 [==============================] - 52s 104ms/step - loss: 0.6700 - acc: 0.8411 - val_loss: 0.8738 - val_acc: 0.7950
Epoch 65/250
500/500 [==============================] - 52s 104ms/step - loss: 0.6709 - acc: 0.8432 - val_loss: 0.9994 - val_acc: 0.7644
Epoch 66/250
500/500 [==============================] - 52s 104ms/step - loss: 0.6766 - acc: 0.8412 - val_loss: 0.9522 - val_acc: 0.7642
Epoch 67/250
500/500 [==============================] - 53s 105ms/step - loss: 0.6728 - acc: 0.8418 - val_loss: 0.6973 - val_acc: 0.8368
Epoch 68/250
500/500 [===================

500/500 [==============================] - 52s 104ms/step - loss: 0.4063 - acc: 0.9157 - val_loss: 0.4473 - val_acc: 0.9092
Epoch 119/250
lr_changed to 0.01
500/500 [==============================] - 52s 104ms/step - loss: 0.4024 - acc: 0.9168 - val_loss: 0.4747 - val_acc: 0.9025
Epoch 120/250
lr_changed to 0.01
500/500 [==============================] - 55s 110ms/step - loss: 0.3978 - acc: 0.9174 - val_loss: 0.4791 - val_acc: 0.9008
Epoch 121/250
lr_changed to 0.01
500/500 [==============================] - 52s 103ms/step - loss: 0.3935 - acc: 0.9198 - val_loss: 0.4757 - val_acc: 0.9012
Epoch 122/250
lr_changed to 0.01
500/500 [==============================] - 52s 104ms/step - loss: 0.3913 - acc: 0.9180 - val_loss: 0.5062 - val_acc: 0.8912
Epoch 123/250
lr_changed to 0.01
500/500 [==============================] - 52s 104ms/step - loss: 0.3863 - acc: 0.9188 - val_loss: 0.4480 - val_acc: 0.9045
Epoch 124/250
lr_changed to 0.01
500/500 [==============================] - 52s 104ms/step 

500/500 [==============================] - 52s 105ms/step - loss: 0.2740 - acc: 0.9441 - val_loss: 0.3852 - val_acc: 0.9170
Epoch 171/250
lr_changed to 0.0005
500/500 [==============================] - 52s 104ms/step - loss: 0.2743 - acc: 0.9429 - val_loss: 0.3846 - val_acc: 0.9182
Epoch 172/250
lr_changed to 0.0005
500/500 [==============================] - 52s 104ms/step - loss: 0.2741 - acc: 0.9434 - val_loss: 0.3847 - val_acc: 0.9182
Epoch 173/250
lr_changed to 0.0005
500/500 [==============================] - 54s 109ms/step - loss: 0.2707 - acc: 0.9436 - val_loss: 0.3838 - val_acc: 0.9189
Epoch 174/250
lr_changed to 0.0005
500/500 [==============================] - 52s 105ms/step - loss: 0.2695 - acc: 0.9434 - val_loss: 0.3807 - val_acc: 0.9187
Epoch 175/250
lr_changed to 0.0005
500/500 [==============================] - 52s 103ms/step - loss: 0.2672 - acc: 0.9461 - val_loss: 0.3794 - val_acc: 0.9194
Epoch 176/250
lr_changed to 0.0005
500/500 [==============================] - 52s

Epoch 222/250
lr_changed to 0.0001
500/500 [==============================] - 52s 103ms/step - loss: 0.2599 - acc: 0.9471 - val_loss: 0.3794 - val_acc: 0.9206
Epoch 223/250
lr_changed to 0.0001
500/500 [==============================] - 52s 103ms/step - loss: 0.2623 - acc: 0.9455 - val_loss: 0.3781 - val_acc: 0.9207
Epoch 224/250
lr_changed to 0.0001
500/500 [==============================] - 52s 103ms/step - loss: 0.2603 - acc: 0.9468 - val_loss: 0.3778 - val_acc: 0.9206
Epoch 225/250
lr_changed to 0.0001
500/500 [==============================] - 52s 103ms/step - loss: 0.2583 - acc: 0.9471 - val_loss: 0.3787 - val_acc: 0.9206
Epoch 226/250
lr_changed to 0.0001
500/500 [==============================] - 52s 103ms/step - loss: 0.2592 - acc: 0.9465 - val_loss: 0.3792 - val_acc: 0.9206
Epoch 227/250
lr_changed to 0.0001
500/500 [==============================] - 54s 109ms/step - loss: 0.2595 - acc: 0.9478 - val_loss: 0.3786 - val_acc: 0.9208
Epoch 228/250
lr_changed to 0.0001
500/500 [==

In [15]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 4s 398us/step
Test loss: 0.3783926488161087
Test accuracy: 0.9208


In [16]:

# Save the trained weights in to .h5 format
model.save("drive/4A1.h5",overwrite='true')
print("Saved model to disk")

Saved model to disk
